<a href="https://colab.research.google.com/github/NIC-DE/Simple_Chatbox_AI_Project_3/blob/main/Simple_Chatbox_AI_Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Simple Chatbot (Rule-based + Retrieval)

A simple chatbot built in Google Colab that combines:
- rule-based intent handling
- retrieval-based answers using TF-IDF
- short-term conversation memory


In [1]:
# Install required library (only once per session)
!pip -q install scikit-learn


In [2]:
# Core Python utilities
import re
from collections import deque

# Text vectorization and similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [3]:
# Mini knowledge base: (question, answer)
KB = [
    ("what is this project",
     "This is a simple chatbot using rule-based intent handling and TF-IDF retrieval."),

    ("how does it work",
     "The bot first checks for simple intents. If none match, it retrieves the most relevant answer using cosine similarity."),

    ("what can you do",
     "I can answer questions from my FAQ, respond to greetings, and keep short-term memory."),

    ("can you recommend a movie",
     "Sure. Tell me your preferred genre and mood.")
]


In [4]:
# Separate questions and answers
questions = [q for q, a in KB]
answers = [a for q, a in KB]

# Vectorize questions using TF-IDF
vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(questions)

def retrieve_answer(user_text, threshold=0.25):
    """
    Finds the most relevant answer from the knowledge base.
    Returns None if similarity is below threshold.
    """
    user_vec = vectorizer.transform([user_text])
    similarities = cosine_similarity(user_vec, X)[0]

    best_index = similarities.argmax()
    best_score = similarities[best_index]

    if best_score < threshold:
        return None, best_score

    return answers[best_index], best_score


In [5]:
def normalize(text: str) -> str:
    """
    Cleans and normalizes user input.
    """
    text = text.lower().strip()
    text = re.sub(r"\s+", " ", text)
    return text


def rule_based(text: str):
    """
    Handles simple intents such as greetings, help, and exit.
    """
    t = normalize(text)

    greetings = {"hi", "hello", "hey"}
    goodbyes = {"bye", "exit", "quit"}

    if t in greetings:
        return "Hello! Ask me something or type 'help'."

    if t in goodbyes:
        return "__EXIT__"

    if "help" in t:
        return "Try asking: 'what is this project', 'how does it work', 'what can you do'."

    return None


In [6]:
# Conversation memory (last N messages)
memory = deque(maxlen=6)

def add_to_memory(role, text):
    memory.append((role, text))

def show_memory():
    if not memory:
        return "Memory is empty."
    return "\n".join([f"{role}: {text}" for role, text in memory])


In [7]:
print("Simple Chatbot is running. Type 'help' or 'exit'.\n")

while True:
    user_input = input("You: ").strip()

    if not user_input:
        continue

    add_to_memory("User", user_input)

    # Step 1: rule-based logic
    rule_response = rule_based(user_input)

    if rule_response == "__EXIT__":
        print("Bot: Goodbye!")
        break

    if rule_response:
        print("Bot:", rule_response)
        add_to_memory("Bot", rule_response)
        continue

    # Step 2: retrieval-based logic
    answer, score = retrieve_answer(user_input)

    if answer:
        bot_response = f"{answer} (confidence={score:.2f})"
    else:
        bot_response = "I'm not sure. Try typing 'help'."

    print("Bot:", bot_response)
    add_to_memory("Bot", bot_response)


Simple Chatbot is running. Type 'help' or 'exit'.

You: help
Bot: Try asking: 'what is this project', 'how does it work', 'what can you do'.
You: can you recommend a movie
Bot: Sure. Tell me your preferred genre and mood. (confidence=1.00)
You: exit
Bot: Goodbye!


In [8]:
print(show_memory())


User: help
Bot: Try asking: 'what is this project', 'how does it work', 'what can you do'.
User: can you recommend a movie
Bot: Sure. Tell me your preferred genre and mood. (confidence=1.00)
User: exit
